# Improving Predictions

Now that we have deployed working models predicting flight delays, it is time to “make believe” that our prediction has proven useful based on user feedback, and further that the prediction is valuable enough that prediction quality is important. In this case, it is time to iteratively improve the quality of our prediction. If a prediction is valuable enough, this becomes a full-time job for one or more people.

In this chapter we will tune our Spark ML classifier and also do additional feature engineering to improve prediction quality. In doing so, we will show you how to iteratively improve predictions.

## Fixing Our Prediction Problem

At this point we realized that our model was always predicting one class, no matter the input. We began by investigating that in a Jupyter notebook at [ch09/Debugging Prediction Problems.ipynb](Debugging Prediction Problems.ipynb).

The notebook itself is very long, and we tried many things to fix our model. It turned out we had made a mistake. We were using `OneHotEncoder` on top of the output of `StringIndexerModel` when we were encoding our nominal/categorical string features. This is how you should encode features for models other than decision trees, but it turns out that for decision tree models, you are supposed to take the string indexes from `StringIndexerModel` and directly compose them with your continuous/numeric features in a `VectorAssembler`. Decision trees are able to infer the fact that indexes represent categories. One benefit of directly adding StringIndexes to your feature vectors is that you then get easily interpretable feature importances.

When we discovered this, we had to go back and edit the book so that we didn’t teach something that was wrong, and so this is now what you see. We thought it worthwhile to link to the notebook, though, to show how this really works in the wild: you build broken shit and then fix it.

## When to Improve Predictions

Not all predictions should be improved. Often something fast and crude will work well enough as an MVP (minimum viable product). Only predictions that prove useful should be improved. It is possible to sink large volumes of time into improving the quality of a prediction, so it is essential that you connect with users before getting sucked into this task. This is why we’ve included the discussion of improving predictions in its own chapter.

## Improving Prediction Performance

There are a few ways to improve an existing predictive model. The first is by tuning the parameters of the statistical model making your prediction. The second is feature engineering.

Tuning model hyperparameters to improve predictive model quality can be done by intuition, or by brute force through something called a grid or random search. We’re going to focus on feature engineering, as hyperparameter tuning is covered elsewhere. A good guide to hyperparameter tuning is available in the Spark documentation on model selection and tuning.

As we move through this chapter, we’ll be using the work we’ve done so far to perform feature engineering. Feature engineering is the most important part of making good predictions. It involves using what you’ve discovered about the data through exploratory data analysis in order to feed your machine learning algorithm better, more consequential data as input.

### Experimental Adhesion Method: See What Sticks

There are several ways to decide which features to use, and Saurav Kaushik has written a post on Analytics Vidhya that introduces them well. The method we employ primarily, which we jokingly entitle the Experimental Adhesion Method, is to quickly select all the features that we can simply compute, and try them all using a random forest or gradient boosted decision tree model (note that even if our application requires another type of model, we still use decision trees to guide feature selection). Then we train the model and inspect the model’s feature importances to “see what sticks.” The most important variables are retained, and this forms the basic model we begin with.

Feature engineering is an iterative process. Based on the feature importances, we ponder what new things we might try using the data we have available. We start with the simplest idea, or the one that is easiest to implement. If the feature importances indicate one type of feature is important, and we can’t easily compute new features similar to this one, we think about how we might acquire new data to join to our training data to use as features.

The key is to be logical and systematic in our exploration of the feature space. You should think about how easy a potential feature is to compute, as well as what it would teach you if it turned out to be important. Are there other, similar features that you could try if this candidate worked? Develop hypotheses and test them in the form of new features. Evaluate each new feature in an experiment and reflect on what you’ve learned before engineering the next feature.

### Establishing Rigorous Metrics for Experiments

In order to improve our classification model, we need to reliably determine its prediction quality in the first place. To do so, we need to beef up our cross-validation code, and then establish a baseline of quality for the original model. Check out [ch09/baseline_spark_mllib_model.py](baseline_spark_mllib_model.py), which we copied from [ch09/train_spark_mllib_model.py](train_spark_mllib_model.py) and altered to improve its cross-validation code.

In order to evaluate the prediction quality of our classifier, we need to use more than one metric. Spark ML’s `MulticlassClassificationEvaluator` offers four metrics: accuracy, weighted precision, weighted recall, and f1.

### Defining Our Classification Metrics

The raw _accuracy_ is just what it sounds like: the number of correct predictions divided by the number of predictions. This is something to check first, but it isn’t adequate alone. _Precision_ is a measure of how useful the result is. _Recall_ describes how complete the results are. The _f1_ score incorporates both precision and recall to determine overall quality. Taken together, the changes to these metrics between consecutive runs of training our model can give us a clear picture of what is happening with our model in terms of prediction quality. We will use these metrics along with feature importance to guide our feature engineering efforts.

### Feature Importance

Model quality metrics aren’t enough to guide the iterative improvements of our model. To understand what is going on with each new run, we need to employ a type of model called a decision tree.

In Spark ML, the best general-purpose multiclass classification model is an implementation of a random forest, the RandomForestClassificationModel, fit by the RandomForestClassifier. Random forests can classify or regress, and they have an important feature that helps us interrogate predictive models through a feature called feature importance.

The importance of a feature is what it sounds like: a measure of how important that feature was in contributing to the accuracy of the model. This information is incredibly useful, as it can serve as a guiding hand to feature engineering. In other words, if you know how important a feature is, you can use this clue to make changes that increase the accuracy of the model, such as removing unimportant features and trying to engineer features similar to those that are most important. Feature engineering is a major theme of Agile Data Science, and it is a big part of why we’ve been doing iterative visualization and exploration (the purpose of which is to shed light on and drive feature engineering).

Note that the state of the art for many classification and regression tasks is a gradient boosted decision tree, but as of version 2.1.0 Spark ML’s implementation—the `GBTClassificationModel`, which is fit by the `GBTClassifier`—can only do binary classification.

### Getting Ready for Experiments

We need to run through the model's code from chapter 8 before we can setup and run an experiment.

In [1]:
import sys, os, re
import json
import datetime, iso8601
from tabulate import tabulate

# Initialize PySpark
APP_NAME = "Improving Predictions"

# If there is no SparkSession, create the environment
try:
    sc and spark
except NameError as e:
    import findspark
    findspark.init()
    import pyspark
    import pyspark.sql

    sc = pyspark.SparkContext()
    spark = pyspark.sql.SparkSession(sc).builder.appName(APP_NAME).getOrCreate()

print("PySpark initialized...")

PySpark initialized...


In [2]:
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType, DateType, TimestampType
from pyspark.sql.types import StructType, StructField
from pyspark.sql.functions import udf

schema = StructType([
    StructField("ArrDelay", DoubleType(), True),     # "ArrDelay":5.0
    StructField("CRSArrTime", TimestampType(), True),    # "CRSArrTime":"2015-12-31T03:20:00.000-08:00"
    StructField("CRSDepTime", TimestampType(), True),    # "CRSDepTime":"2015-12-31T03:05:00.000-08:00"
    StructField("Carrier", StringType(), True),     # "Carrier":"WN"
    StructField("DayOfMonth", IntegerType(), True), # "DayOfMonth":31
    StructField("DayOfWeek", IntegerType(), True),  # "DayOfWeek":4
    StructField("DayOfYear", IntegerType(), True),  # "DayOfYear":365
    StructField("DepDelay", DoubleType(), True),     # "DepDelay":14.0
    StructField("Dest", StringType(), True),        # "Dest":"SAN"
    StructField("Distance", DoubleType(), True),     # "Distance":368.0
    StructField("FlightDate", DateType(), True),    # "FlightDate":"2015-12-30T16:00:00.000-08:00"
    StructField("FlightNum", StringType(), True),   # "FlightNum":"6109"
    StructField("Origin", StringType(), True),      # "Origin":"TUS"
])

input_path = "../data/simple_flight_delay_features.json"
features = spark.read.json(input_path, schema=schema)

# Sample 10% to make executable inside the notebook
features = features.sample(False, 0.1)

features.first()

Row(ArrDelay=-5.0, CRSArrTime=datetime.datetime(2015, 1, 19, 0, 4), CRSDepTime=datetime.datetime(2015, 1, 18, 23, 0), Carrier='B6', DayOfMonth=18, DayOfWeek=7, DayOfYear=18, DepDelay=0.0, Dest='BOS', Distance=187.0, FlightDate=datetime.date(2015, 1, 18), FlightNum='718', Origin='JFK')

In [3]:
#
# Check for nulls in features before using Spark ML
#
null_counts = [(column, features.where(features[column].isNull()).count()) for column in features.columns]
cols_with_nulls = filter(lambda x: x[1] > 0, null_counts)
print("Columns with nulls that need to be filtered: {}".format(
    str(list(cols_with_nulls))
))

Columns with nulls that need to be filtered: []


In [4]:
#
# Add a Route variable to replace FlightNum
#
from pyspark.sql.functions import lit, concat
features_with_route = features.withColumn(
  'Route',
  concat(
    features.Origin,
    lit('-'),
    features.Dest
  )
)
features_with_route.show(3)

+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+----------+---------+------+-------+
|ArrDelay|         CRSArrTime|         CRSDepTime|Carrier|DayOfMonth|DayOfWeek|DayOfYear|DepDelay|Dest|Distance|FlightDate|FlightNum|Origin|  Route|
+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+----------+---------+------+-------+
|    -5.0|2015-01-19 00:04:00|2015-01-18 23:00:00|     B6|        18|        7|       18|     0.0| BOS|   187.0|2015-01-18|      718|   JFK|JFK-BOS|
|     1.0|2015-01-19 04:40:00|2015-01-18 23:59:00|     B6|        18|        7|       18|     3.0| BQN|  1576.0|2015-01-18|      839|   JFK|JFK-BQN|
|   -12.0|2015-01-18 16:12:00|2015-01-18 13:59:00|     B6|        18|        7|       18|    -6.0| CHS|   636.0|2015-01-18|     1373|   JFK|JFK-CHS|
+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+---

In [5]:
#
# Use pysmark.ml.feature.Bucketizer to bucketize ArrDelay into on-time, slightly late, very late (0, 1, 2)
#
from pyspark.ml.feature import Bucketizer

# Setup the Bucketizer
splits = [-float("inf"), -15.0, 0, 30.0, float("inf")]
arrival_bucketizer = Bucketizer(
  splits=splits,
  inputCol="ArrDelay",
  outputCol="ArrDelayBucket"
)

# Save the model
arrival_bucketizer_path = "../models/arrival_bucketizer_2.0.bin"
arrival_bucketizer.write().overwrite().save(arrival_bucketizer_path)

# Apply the model
ml_bucketized_features = arrival_bucketizer.transform(features_with_route)
ml_bucketized_features.select("ArrDelay", "ArrDelayBucket").show(5)

+--------+--------------+
|ArrDelay|ArrDelayBucket|
+--------+--------------+
|    -5.0|           1.0|
|     1.0|           2.0|
|   -12.0|           1.0|
|    29.0|           2.0|
|    10.0|           2.0|
+--------+--------------+
only showing top 5 rows



In [6]:
#
# Extract features tools in with pyspark.ml.feature
#
from pyspark.ml.feature import StringIndexer, VectorAssembler

# Turn category fields into indexes
for column in ["Carrier", "DayOfMonth", "DayOfWeek", "DayOfYear",
             "Origin", "Dest", "Route"]:
    
    print("Indexing column \"{}\" ...".format(column))
    
    string_indexer = StringIndexer(
      inputCol=column,
      outputCol=column + "_index"
    )

    string_indexer_model = string_indexer.fit(ml_bucketized_features)
    ml_bucketized_features = string_indexer_model.transform(ml_bucketized_features)

    # Drop the original column
    ml_bucketized_features = ml_bucketized_features.drop(column)

    # Save the pipeline model
    string_indexer_output_path = "../models/string_indexer_model_{}.bin".format(
      column
    )
    string_indexer_model.write().overwrite().save(string_indexer_output_path)

print("Indexed all string columns!")

Indexing column "Carrier" ...
Indexing column "DayOfMonth" ...
Indexing column "DayOfWeek" ...
Indexing column "DayOfYear" ...
Indexing column "Origin" ...
Indexing column "Dest" ...
Indexing column "Route" ...
Indexed all string columns!


In [7]:
# Handle continuous, numeric fields by combining them into one feature vector
numeric_columns = ["DepDelay", "Distance"]
index_columns = ["Carrier_index", "DayOfMonth_index",
                 "DayOfWeek_index", "DayOfYear_index", "Origin_index",
                 "Origin_index", "Dest_index", "Route_index"]
vector_assembler = VectorAssembler(
    inputCols=numeric_columns + index_columns,
    outputCol="Features_vec"
)
final_vectorized_features = vector_assembler.transform(ml_bucketized_features)

# Save the numeric vector assembler
vector_assembler_path = "../models/numeric_vector_assembler.bin"
vector_assembler.write().overwrite().save(vector_assembler_path)

# Drop the index columns
for column in index_columns:
    final_vectorized_features = final_vectorized_features.drop(column)

# Inspect the finalized features
final_vectorized_features.show(5)

+--------+-------------------+-------------------+--------+--------+----------+---------+--------------+--------------------+
|ArrDelay|         CRSArrTime|         CRSDepTime|DepDelay|Distance|FlightDate|FlightNum|ArrDelayBucket|        Features_vec|
+--------+-------------------+-------------------+--------+--------+----------+---------+--------------+--------------------+
|    -5.0|2015-01-19 00:04:00|2015-01-18 23:00:00|     0.0|   187.0|2015-01-18|      718|           1.0|[0.0,187.0,8.0,25...|
|     1.0|2015-01-19 04:40:00|2015-01-18 23:59:00|     3.0|  1576.0|2015-01-18|      839|           2.0|[3.0,1576.0,8.0,2...|
|   -12.0|2015-01-18 16:12:00|2015-01-18 13:59:00|    -6.0|   636.0|2015-01-18|     1373|           1.0|[-6.0,636.0,8.0,2...|
|    29.0|2015-01-18 19:17:00|2015-01-18 17:13:00|    41.0|   541.0|2015-01-18|     1119|           2.0|[41.0,541.0,8.0,2...|
|    10.0|2015-01-19 00:18:00|2015-01-18 21:29:00|    14.0|  2248.0|2015-01-18|      611|           2.0|[14.0,2248.0,8

### Implementing A More Rigorous Experiment

In order to be confident in our experiment for each measure, we need to repeat it at least twice to see how it varies. This is the degree to which we cross-validate. In addition, we need to loop and run the measurement code once for each score. Once we’ve collected several scores for each metric, we look at both the average and standard deviation for each score. Taken together, these scores give us a picture of the quality of our classifier.

To begin, we need to iterate and repeat our experiment N times. For each experiment we need to compute a test/train split, then we need to train the model on the training data and apply it to the test data. Then we use `MulticlassClassificationEvaluator` to get a score, once for each metric. We gather the scores in a list for each metric, which we will evaluate at the end of the experiment:

In [8]:
#
# Cross validate, train and evaluate classifier: loop 5 times for 4 metrics
#

from collections import defaultdict
scores = defaultdict(list)
metric_names = ["accuracy", "weightedPrecision", "weightedRecall", "f1"]
split_count = 3

for i in range(1, split_count + 1):
  print("Run {} out of {} of test/train splits in cross validation...".format(
      i,
      split_count,
    )
  )

  # Test/train split
  training_data, test_data = final_vectorized_features.randomSplit([0.8, 0.2])

  # Instantiate and fit random forest classifier on all the data
  from pyspark.ml.classification import RandomForestClassifier
  rfc = RandomForestClassifier(
    featuresCol="Features_vec",
    labelCol="ArrDelayBucket",
    predictionCol="Prediction",
    maxBins=4657,
    maxMemoryInMB=1024
  )
  model = rfc.fit(training_data)

  # Save the new model over the old one
  model_output_path = "../models/spark_random_forest_classifier.flight_delays.baseline.bin"
  model.write().overwrite().save(model_output_path)

  # Evaluate model using test data
  predictions = model.transform(test_data)
  
  # Evaluate this split's results for each metric
  from pyspark.ml.evaluation import MulticlassClassificationEvaluator
  for metric_name in metric_names:
    
    evaluator = MulticlassClassificationEvaluator(
      labelCol="ArrDelayBucket",
      predictionCol="Prediction",
      metricName=metric_name
    )
    score = evaluator.evaluate(predictions)

    scores[metric_name].append(score)
    print("{} = {}".format(metric_name, score))

Run 1 out of 3 of test/train splits in cross validation...
accuracy = 0.5836824696802646
weightedPrecision = 0.6997762781243368
weightedRecall = 0.5836824696802646
f1 = 0.5179702041608492
Run 2 out of 3 of test/train splits in cross validation...
accuracy = 0.5825253295565965
weightedPrecision = 0.6494016935831022
weightedRecall = 0.5825253295565966
f1 = 0.5170259360198566
Run 3 out of 3 of test/train splits in cross validation...
accuracy = 0.581765087605451
weightedPrecision = 0.6004499844248835
weightedRecall = 0.581765087605451
f1 = 0.5159156166131214


## Processing Run Results

Our run leaves us with a `defaultdict` of scores, with one list for each metric. Now we need to compute the average and standard deviation of each list to give us the overall average and standard deviation of each metric:

Note that we need to compute both the average and standard deviation of each metric from our run. The average will tell us the approximate performance level, and the standard deviation will tell us how much a model's performance varies. Less variance is desirable. We'll use this information in tuning our model.

In [9]:
#
# Evaluate average and STD of each metric and print a table
#
import numpy as np
score_averages = defaultdict(float)

# Compute the table data
average_stds = [] # ha
for metric_name in metric_names:
  metric_scores = scores[metric_name]
  
  average_accuracy = sum(metric_scores) / len(metric_scores)
  score_averages[metric_name] = average_accuracy
  
  std_accuracy = np.std(metric_scores)
  
  average_stds.append((metric_name, average_accuracy, std_accuracy))

# Print the table
print("\nExperiment Log")
print("--------------")
print(tabulate(average_stds, headers=["Metric", "Average", "STD"]))


Experiment Log
--------------
Metric               Average          STD
-----------------  ---------  -----------
accuracy            0.582658  0.000788338
weightedPrecision   0.649876  0.0405512
weightedRecall      0.582658  0.000788338
f1                  0.516971  0.000839694


The standard deviations indicate that we might not even need to perform k-fold cross-validation, but an inspection of the underlying scores says otherwise:

In [10]:
scores

defaultdict(list,
            {'accuracy': [0.5836824696802646,
              0.5825253295565965,
              0.581765087605451],
             'weightedPrecision': [0.6997762781243368,
              0.6494016935831022,
              0.6004499844248835],
             'weightedRecall': [0.5836824696802646,
              0.5825253295565966,
              0.581765087605451],
             'f1': [0.5179702041608492,
              0.5170259360198566,
              0.5159156166131214]})

There is actually significant variation between runs, and this could obscure a small improvement (or degradation) in prediction quality.

The iterations take time, and this discourages experimentation. A middle ground should be found.

## Comparing Experiments to Determine Improvements

Now that we have our baseline metrics, we can repeat this code as we improve the model and see what the effect is in terms of the four metrics available to us. So it seems we are done, that we can start playing with our model and features. However, we will quickly run into a problem. We will lose track of the score from the previous run, printed on the screen above many logs for each run, unless we write it down each time. And this is tedious. So, we need to automate this process.

What we need to do is load a score log from disk, evaluate the current score in terms of the previous one, and store a new entry to the log back to disk for the next run to access. The following code achieves this aim.

First we use pickle to load any existing score log. If this is not present, we initialize a new log, which is simply an empty Python list. Next we prepare the new log entry—a simple Python dict containing the average score for each of four metrics. Then we subtract the previous run’s score to determine the change in this run. This is the information we use to evaluate whether our change worked or not (along with any changes in feature importances, which we will address as well). 

Finally, we append the new score entry to the log and store it back to disk:

In [11]:
#
# Persist the score to a sccore log that exists between runs
#
import pickle

# Load the score log or initialize an empty one
try:
  score_log_filename = "../models/score_log.pickle"
  score_log = pickle.load(open(score_log_filename, "rb"))
  if not isinstance(score_log, list):
    score_log = []
except IOError:
  score_log = []

# Compute the existing score log entry
score_log_entry = {metric_name: score_averages[metric_name] for metric_name in metric_names}

# Compute and display the change in score for each metric
try:
  last_log = score_log[-1]
except (IndexError, TypeError, AttributeError):
  last_log = score_log_entry

experiment_report = []
for metric_name in metric_names:
  run_delta = score_log_entry[metric_name] - last_log[metric_name]
  experiment_report.append((metric_name, run_delta))

print("\nExperiment Report")
print("-----------------")
print(tabulate(experiment_report, headers=["Metric", "Score"]))

# Append the existing average scores to the log
score_log.append(score_log_entry)

# Persist the log for next run
pickle.dump(score_log, open(score_log_filename, "wb"))


Experiment Report
-----------------
Metric                   Score
-----------------  -----------
accuracy           -0.0114008
weightedPrecision  -0.00396363
weightedRecall     -0.0114008
f1                 -0.0216341


Now when we run our script, we will get a report that shows the change between this run and the last run. We can use this, along with our feature importances, to direct our efforts at improving the model. For instance, an example test run shows the model accuracy increase by .003:

```
Experiment Report
-----------------
Metric                   Score
-----------------  -----------
accuracy            0.00300548
weightedPrecision  -0.00592227
weightedRecall      0.00300548
f1                 -0.0105553
```

Jump back to the code for the model, the code under the section `Implementing a More Rigorous Experiment`. Re-run all the code between there and here, the last three code blocks. See how the score changed slightly? You will use these changes to guide you as you change the model!

## Inspecting Changes in Feature Importance

We can use the list of columns given to our final `VectorAssembler` along with `RandomForestClassificationModel.featureImportances` to derive the importance of each named feature. This is extremely valuable, because like with our prediction quality scores, we can look at changes in feature importances for all features between runs. If a newly introduced feature turns out to be important, it is usually worth adding to the model, so long as it doesn’t hurt quality.

### Logging Feature Importances

We begin by altering our experiment loop to record feature importances for each run. Check out the abbreviated content from [ch09/improved_spark_mllib_model.py](improved_spark_mllib_model.py):

In [12]:
#
# Cross validate, train and evaluate classifier: loop 5 times for 4 metrics
#

from collections import defaultdict
scores = defaultdict(list)
feature_importances = defaultdict(list)
metric_names = ["accuracy", "weightedPrecision", "weightedRecall", "f1"]
split_count = 3

for i in range(1, split_count + 1):
  print("\nRun {} out of {} of test/train splits in cross validation...".format(
      i,
      split_count,
    )
  )

  # Test/train split
  training_data, test_data = final_vectorized_features.randomSplit([0.8, 0.2])

  # Instantiate and fit random forest classifier on all the data
  from pyspark.ml.classification import RandomForestClassifier
  rfc = RandomForestClassifier(
    featuresCol="Features_vec",
    labelCol="ArrDelayBucket",
    predictionCol="Prediction",
    maxBins=4657,
  )
  model = rfc.fit(training_data)

  # Save the new model over the old one
  model_output_path = "../models/spark_random_forest_classifier.flight_delays.baseline.bin"
  model.write().overwrite().save(model_output_path)

  # Evaluate model using test data
  predictions = model.transform(test_data)

  # Evaluate this split's results for each metric
  from pyspark.ml.evaluation import MulticlassClassificationEvaluator
  for metric_name in metric_names:
    evaluator = MulticlassClassificationEvaluator(
      labelCol="ArrDelayBucket",
      predictionCol="Prediction",
      metricName=metric_name
    )
    score = evaluator.evaluate(predictions)
  
    scores[metric_name].append(score)
    print("{} = {}".format(metric_name, score))

  #
  # Collect feature importances
  #
  feature_names = vector_assembler.getInputCols()
  feature_importance_list = model.featureImportances
  for feature_name, feature_importance in zip(feature_names, feature_importance_list):
    feature_importances[feature_name].append(feature_importance)


Run 1 out of 3 of test/train splits in cross validation...
accuracy = 0.580483457725298
weightedPrecision = 0.6361932319624914
weightedRecall = 0.580483457725298
f1 = 0.5146409852564

Run 2 out of 3 of test/train splits in cross validation...
accuracy = 0.5689097198973329
weightedPrecision = 0.6384506615016216
weightedRecall = 0.568909719897333
f1 = 0.5048858150091734

Run 3 out of 3 of test/train splits in cross validation...
accuracy = 0.5882746321890325
weightedPrecision = 0.659108261496373
weightedRecall = 0.5882746321890326
f1 = 0.526289877315701


### Inspecting Feature Importances

Next, we need to compute the average of the importance for each feature. Note that we use a `defaultdict(float)` to ensure that accessing empty keys returns zero. This will be important when comparing entries in the log with different sets of features. In order to print the feature importances, we need to sort them first, by descending order of importance:

In [13]:
#
# Analyze and report feature importance changes
#

# Compute averages for each feature
feature_importance_entry = defaultdict(float)
for feature_name, value_list in feature_importances.items():
  average_importance = sum(value_list) / len(value_list)
  feature_importance_entry[feature_name] = average_importance

# Sort the feature importances in descending order and print
import operator
sorted_feature_importances = sorted(
  feature_importance_entry.items(),
  key=operator.itemgetter(1),
  reverse=True
)

print("\nFeature Importances")
print("-------------------")
print(tabulate(sorted_feature_importances, headers=['Name', 'Importance']))


Feature Importances
-------------------
Name                Importance
----------------  ------------
DepDelay            0.803014
Route_index         0.0432006
DayOfYear_index     0.041677
Origin_index        0.0225873
Dest_index          0.0211217
DayOfMonth_index    0.0186109
Distance            0.0168524
Carrier_index       0.00705917
DayOfWeek_index     0.00328912


### Feature Importance Differences Between Runs

Next we need to perform the same housekeeping as we did for the model score log: load the model, create an entry for this experiment, load the last experiment and compute the change for each feature between that experiment and the current one, and then print a report on these deltas.

First we load the last feature log. If it isn’t available because it doesn’t exist, we initialize the last_feature_log with zeros for each feature, so that new features will have a positive score equal to their amount:

In [14]:
#
# Compare this run's feature importances with the previous run's
#
  
# Load the feature importance log or initialize an empty one
try:
  feature_log_filename = "../models/feature_log.pickle"
  feature_log = pickle.load(open(feature_log_filename, "rb"))
  if not isinstance(feature_log, list):
    feature_log = []
except IOError:
  feature_log = []

# Compute and display the change in score for each feature
try:
  last_feature_log = feature_log[-1]
except (IndexError, TypeError, AttributeError):
  last_feature_log = defaultdict(float)
  for feature_name, importance in feature_importance_entry.items():
    last_feature_log[feature_name] = importance

Next we compute the change between the last run and the current one:

In [15]:
# Compute the deltas
feature_deltas = {}
for feature_name in feature_importances.keys():
  run_delta = feature_importance_entry[feature_name] - last_feature_log[feature_name]
  feature_deltas[feature_name] = run_delta

In order to display them, we need to sort the feature importance changes in descending order, to show the biggest change first:

In [16]:
# Sort feature deltas, biggest change first
import operator
sorted_feature_deltas = sorted(
  feature_deltas.items(),
  key=operator.itemgetter(1),
  reverse=True
)

Then we display the sorted feature deltas:

In [17]:
# Display sorted feature deltas
print("\nFeature Importance Delta Report")
print("-------------------------------")
print(tabulate(sorted_feature_deltas, headers=["Feature", "Delta"]))


Feature Importance Delta Report
-------------------------------
Feature                  Delta
----------------  ------------
DayOfYear_index    0.0184643
Route_index        0.00612603
DayOfWeek_index    0.00185113
Dest_index         0.00118472
Carrier_index      0.000929297
Origin_index      -0.000450557
Distance          -0.00469211
DayOfMonth_index  -0.0109353
DepDelay          -0.0120269


Finally, as with the score log, we append our entry to the log and save it for the next run:

In [18]:
# Append the existing average deltas to the log
feature_log.append(feature_importance_entry)

# Persist the log for next run
pickle.dump(feature_log, open(feature_log_filename, "wb"))

We’ll use the raw feature importances as well as the changes in feature importance to guide our creation or alteration of features as we improve the model.

### Conclusion

Now that we have the ability to understand the effect of changes between experimental runs, we can detect changes that improve our model. We can start adding features to test their effect on the model’s prediction quality, and pursue related features that help improve quality! Without this setup, we would be hard put to make positive changes. With it, we are only bounded by our creativity in our efforts to improve the model.

## Time of Day as a Feature

In examining our feature importances, it looks like the date/time fields have some impact. What if we extracted the hour/minute as an integer from the datetime for departure/arrival fields? This would inform the model about morning versus afternoon versus red-eye flights, which surely affects on-time performance, as there is more traffic in the morning than overnight.

Check out [ch09/explore_delays.py](explore_delays.py). Let’s start by exploring the premise of this feature, that lateness varies by the time of day of the flight:

In [19]:
features.registerTempTable("features")
features.show(5)

+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+----------+---------+------+
|ArrDelay|         CRSArrTime|         CRSDepTime|Carrier|DayOfMonth|DayOfWeek|DayOfYear|DepDelay|Dest|Distance|FlightDate|FlightNum|Origin|
+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+----------+---------+------+
|    -5.0|2015-01-19 00:04:00|2015-01-18 23:00:00|     B6|        18|        7|       18|     0.0| BOS|   187.0|2015-01-18|      718|   JFK|
|     1.0|2015-01-19 04:40:00|2015-01-18 23:59:00|     B6|        18|        7|       18|     3.0| BQN|  1576.0|2015-01-18|      839|   JFK|
|   -12.0|2015-01-18 16:12:00|2015-01-18 13:59:00|     B6|        18|        7|       18|    -6.0| CHS|   636.0|2015-01-18|     1373|   JFK|
|    29.0|2015-01-18 19:17:00|2015-01-18 17:13:00|     B6|        18|        7|       18|    41.0| CLT|   541.0|2015-01-18|     1119|   JFK|
|    10.0|201

In [20]:
spark.sql("""
  SELECT
    HOUR(CRSDepTime) + 1 AS Hour,
    AVG(ArrDelay),
    STD(ArrDelay)
  FROM features
  GROUP BY HOUR(CRSDepTime)
  ORDER BY HOUR(CRSDepTime)
""").show(24)

+----+-------------------+---------------------+
|Hour|      avg(ArrDelay)|stddev_samp(ArrDelay)|
+----+-------------------+---------------------+
|   1| -2.192982456140351|   20.778971681976973|
|   2|          19.546875|    84.70376571668642|
|   3| 3.4705882352941178|   25.956977980542206|
|   4|                0.5|   32.439687626938294|
|   5| 20.636363636363637|    43.18627728173089|
|   6| -1.220421393841167|    36.55542009242717|
|   7|          -2.834375|    36.34469164885469|
|   8|-0.3418894830659537|   35.022812659136456|
|   9|  1.416891284815813|    32.13213100339773|
|  10| 2.8312191684284707|   37.540860849727785|
|  11|  5.243448275862069|    36.22973021337575|
|  12|  6.550379914106376|    48.33557880220105|
|  13|  6.205017921146953|   45.426249946539585|
|  14|  7.212092434314656|    38.11610997719809|
|  15|  7.720670391061453|     36.8521055518414|
|  16|   8.17374651810585|    38.19846931930225|
|  17|  8.577387486278814|   39.076879599499335|
|  18|   9.275559883

In [21]:
spark.sql("""
  SELECT
    HOUR(CRSArrTime) + 1 AS Hour,
    AVG(ArrDelay),
    STD(ArrDelay)
  FROM features
  GROUP BY HOUR(CRSArrTime)
  ORDER BY HOUR(CRSArrTime)
""").show(24)

+----+--------------------+---------------------+
|Hour|       avg(ArrDelay)|stddev_samp(ArrDelay)|
+----+--------------------+---------------------+
|   1|   15.05813953488372|    45.11575126573887|
|   2|    8.23076923076923|    33.22606898083233|
|   3| -20.363636363636363|   12.347248497318965|
|   4|              9.6875|   30.466853573460234|
|   5|   8.622641509433961|    36.50301695386831|
|   6|  1.4923664122137406|   32.725813491923155|
|   7|  1.3789473684210527|    41.11709607458835|
|   8| -1.7695252679938744|   30.903453998646935|
|   9| -1.0879017013232515|    36.14097463289268|
|  10|-0.14920140241527075|    31.86169361195121|
|  11|  2.1374072765807135|    39.28125254640734|
|  12|  1.4746765249537892|     32.5702955940238|
|  13|   5.110858995137764|    47.06548162966482|
|  14|  3.0099431818181817|    35.14433937724822|
|  15|    6.77048622917375|   46.418065031775214|
|  16|   6.250175192711983|    40.03300152343383|
|  17|  7.7013546798029555|    38.32877850237684|


In [22]:
from pyspark.sql.functions import hour

features = features.withColumn('CRSDepHourOfDay', hour(features.CRSDepTime))
features = features.withColumn('CRSArrHourOfDay', hour(features.CRSArrTime))

departure_cov = features.stat.cov('CRSDepHourOfDay', 'ArrDelay')
arrival_cov = features.stat.cov('CRSArrHourOfDay', 'ArrDelay')

print("Departure delay covariance: {:,}".format(departure_cov))
print("Arrival delay covariance:   {:,}".format(arrival_cov))

Departure delay covariance: 16.132755399553687
Arrival delay covariance:   15.338965369582844


In [23]:
features.select(
    "CRSDepTime", 
    "CRSDepHourOfDay", 
    "CRSArrTime", 
    "CRSArrHourOfDay"
).show()

+-------------------+---------------+-------------------+---------------+
|         CRSDepTime|CRSDepHourOfDay|         CRSArrTime|CRSArrHourOfDay|
+-------------------+---------------+-------------------+---------------+
|2015-01-18 23:00:00|             23|2015-01-19 00:04:00|              0|
|2015-01-18 23:59:00|             23|2015-01-19 04:40:00|              4|
|2015-01-18 13:59:00|             13|2015-01-18 16:12:00|             16|
|2015-01-18 17:13:00|             17|2015-01-18 19:17:00|             19|
|2015-01-18 21:29:00|             21|2015-01-19 00:18:00|              0|
|2015-01-18 06:30:00|              6|2015-01-18 09:47:00|              9|
|2015-01-18 21:40:00|             21|2015-01-19 00:56:00|              0|
|2015-01-18 19:30:00|             19|2015-01-18 21:18:00|             21|
|2015-01-18 18:59:00|             18|2015-01-18 22:46:00|             22|
|2015-01-18 15:40:00|             15|2015-01-18 19:04:00|             19|
|2015-01-18 22:29:00|             22|2

### Encoding Our New Features

Now we must repeat the feature encoding process such that it includes these new features. Lets take a look at what our features look like at this moment:

In [24]:
features.show(5)

+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+----------+---------+------+---------------+---------------+
|ArrDelay|         CRSArrTime|         CRSDepTime|Carrier|DayOfMonth|DayOfWeek|DayOfYear|DepDelay|Dest|Distance|FlightDate|FlightNum|Origin|CRSDepHourOfDay|CRSArrHourOfDay|
+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+----------+---------+------+---------------+---------------+
|    -5.0|2015-01-19 00:04:00|2015-01-18 23:00:00|     B6|        18|        7|       18|     0.0| BOS|   187.0|2015-01-18|      718|   JFK|             23|              0|
|     1.0|2015-01-19 04:40:00|2015-01-18 23:59:00|     B6|        18|        7|       18|     3.0| BQN|  1576.0|2015-01-18|      839|   JFK|             23|              4|
|   -12.0|2015-01-18 16:12:00|2015-01-18 13:59:00|     B6|        18|        7|       18|    -6.0| CHS|   636.0|2015-01-18|     1373|  

So we're back at the beginning, and still have to add `Route`, bucketize the data, encode our string and numeric fields and then combine them into a single vector. Lets get started!

In [25]:
#
# Add a Route variable to replace FlightNum
#
from pyspark.sql.functions import lit, concat
features_with_route = features.withColumn(
  'Route',
  concat(
    features.Origin,
    lit('-'),
    features.Dest
  )
)
features_with_route.show(6)

+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+----------+---------+------+---------------+---------------+-------+
|ArrDelay|         CRSArrTime|         CRSDepTime|Carrier|DayOfMonth|DayOfWeek|DayOfYear|DepDelay|Dest|Distance|FlightDate|FlightNum|Origin|CRSDepHourOfDay|CRSArrHourOfDay|  Route|
+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+----------+---------+------+---------------+---------------+-------+
|    -5.0|2015-01-19 00:04:00|2015-01-18 23:00:00|     B6|        18|        7|       18|     0.0| BOS|   187.0|2015-01-18|      718|   JFK|             23|              0|JFK-BOS|
|     1.0|2015-01-19 04:40:00|2015-01-18 23:59:00|     B6|        18|        7|       18|     3.0| BQN|  1576.0|2015-01-18|      839|   JFK|             23|              4|JFK-BQN|
|   -12.0|2015-01-18 16:12:00|2015-01-18 13:59:00|     B6|        18|        7|       18|    -6

In [26]:
#
# Use pysmark.ml.feature.Bucketizer to bucketize ArrDelay into on-time, slightly late, very late (0, 1, 2)
#
from pyspark.ml.feature import Bucketizer

# Setup the Bucketizer
splits = [-float("inf"), -15.0, 0, 30.0, float("inf")]
arrival_bucketizer = Bucketizer(
  splits=splits,
  inputCol="ArrDelay",
  outputCol="ArrDelayBucket"
)

# Save the model
arrival_bucketizer_path = "../models/arrival_bucketizer_2.0.bin"
arrival_bucketizer.write().overwrite().save(arrival_bucketizer_path)

# Apply the model
ml_bucketized_features = arrival_bucketizer.transform(features_with_route)
ml_bucketized_features.select("ArrDelay", "ArrDelayBucket").show(5)

+--------+--------------+
|ArrDelay|ArrDelayBucket|
+--------+--------------+
|    -5.0|           1.0|
|     1.0|           2.0|
|   -12.0|           1.0|
|    29.0|           2.0|
|    10.0|           2.0|
+--------+--------------+
only showing top 5 rows



In [27]:
#
# Extract features tools in with pyspark.ml.feature
#
from pyspark.ml.feature import StringIndexer, VectorAssembler

# Turn category fields into indexes
for column in ["Carrier", "DayOfMonth", "DayOfWeek", "DayOfYear",
               "Origin", "Dest", "Route"]:
  string_indexer = StringIndexer(
    inputCol=column,
    outputCol=column + "_index"
  )

  string_indexer_model = string_indexer.fit(ml_bucketized_features)
  ml_bucketized_features = string_indexer_model.transform(ml_bucketized_features)

  # Save the pipeline model
  string_indexer_output_path = "../models/string_indexer_model_3.0.{}.bin".format(
    column
  )
  string_indexer_model.write().overwrite().save(string_indexer_output_path)

ml_bucketized_features.show(5)

+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+----------+---------+------+---------------+---------------+-------+--------------+-------------+----------------+---------------+---------------+------------+----------+-----------+
|ArrDelay|         CRSArrTime|         CRSDepTime|Carrier|DayOfMonth|DayOfWeek|DayOfYear|DepDelay|Dest|Distance|FlightDate|FlightNum|Origin|CRSDepHourOfDay|CRSArrHourOfDay|  Route|ArrDelayBucket|Carrier_index|DayOfMonth_index|DayOfWeek_index|DayOfYear_index|Origin_index|Dest_index|Route_index|
+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+----------+---------+------+---------------+---------------+-------+--------------+-------------+----------------+---------------+---------------+------------+----------+-----------+
|    -5.0|2015-01-19 00:04:00|2015-01-18 23:00:00|     B6|        18|        7|       18|     0.0| BOS|   187.0|201

In [28]:
# Combine continuous, numeric fields with indexes of nominal ones
# ...into one feature vector
numeric_columns = [
  "DepDelay", "Distance",
  "CRSDepHourOfDay", "CRSArrHourOfDay"
]
index_columns = ["Carrier_index", "DayOfMonth_index",
                 "DayOfWeek_index", "DayOfYear_index", "Origin_index",
                 "Origin_index", "Dest_index", "Route_index"]
vector_assembler = VectorAssembler(
  inputCols=numeric_columns + index_columns,
  outputCol="Features_vec"
)
final_vectorized_features = vector_assembler.transform(ml_bucketized_features)

# Save the numeric vector assembler
vector_assembler_path = "../models/numeric_vector_assembler_3.0.bin"
vector_assembler.write().overwrite().save(vector_assembler_path)

# Drop the index columns
for column in index_columns:
  final_vectorized_features = final_vectorized_features.drop(column)

# Inspect the finalized features
final_vectorized_features.show(5)

+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+----------+---------+------+---------------+---------------+-------+--------------+--------------------+
|ArrDelay|         CRSArrTime|         CRSDepTime|Carrier|DayOfMonth|DayOfWeek|DayOfYear|DepDelay|Dest|Distance|FlightDate|FlightNum|Origin|CRSDepHourOfDay|CRSArrHourOfDay|  Route|ArrDelayBucket|        Features_vec|
+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+----------+---------+------+---------------+---------------+-------+--------------+--------------------+
|    -5.0|2015-01-19 00:04:00|2015-01-18 23:00:00|     B6|        18|        7|       18|     0.0| BOS|   187.0|2015-01-18|      718|   JFK|             23|              0|JFK-BOS|           1.0|[0.0,187.0,23.0,0...|
|     1.0|2015-01-19 04:40:00|2015-01-18 23:59:00|     B6|        18|        7|       18|     3.0| BQN|  1576.0|2015-01-18|      839

### Training with Our New Features

Now we will train the model again, noting the performance and feature importances as we did before. This allows us to see the change in performance owing to the introduction of these new fields, `CRSDepHourOfDay` and `CRSArrHourOfDay`.

In [29]:
#
# Cross validate, train and evaluate classifier: loop 5 times for 4 metrics
#

from collections import defaultdict
scores = defaultdict(list)
feature_importances = defaultdict(list)
metric_names = ["accuracy", "weightedPrecision", "weightedRecall", "f1"]
split_count = 3

for i in range(1, split_count + 1):
  print("\nRun {} out of {} of test/train splits in cross validation...".format(
      i,
      split_count,
    )
  )

  # Test/train split
  training_data, test_data = final_vectorized_features.randomSplit([0.8, 0.2])

  # Instantiate and fit random forest classifier on all the data
  from pyspark.ml.classification import RandomForestClassifier
  rfc = RandomForestClassifier(
    featuresCol="Features_vec",
    labelCol="ArrDelayBucket",
    predictionCol="Prediction",
    maxBins=4657,
    maxMemoryInMB=1024
  )
  model = rfc.fit(training_data)

  # Save the new model over the old one
  model_output_path = "../models/spark_random_forest_classifier.flight_delays.baseline.bin"
  model.write().overwrite().save(model_output_path)

  # Evaluate model using test data
  predictions = model.transform(test_data)

  # Evaluate this split's results for each metric
  from pyspark.ml.evaluation import MulticlassClassificationEvaluator
  for metric_name in metric_names:
    evaluator = MulticlassClassificationEvaluator(
      labelCol="ArrDelayBucket",
      predictionCol="Prediction",
      metricName=metric_name
    )
    score = evaluator.evaluate(predictions)
  
    scores[metric_name].append(score)
    print("{} = {}".format(metric_name, score))

  #
  # Collect feature importances
  #
  feature_names = vector_assembler.getInputCols()
  feature_importance_list = model.featureImportances
  for feature_name, feature_importance in zip(feature_names, feature_importance_list):
    feature_importances[feature_name].append(feature_importance)


Run 1 out of 3 of test/train splits in cross validation...
accuracy = 0.5810062056737588
weightedPrecision = 0.6934407918032928
weightedRecall = 0.5810062056737588
f1 = 0.5148110750855828

Run 2 out of 3 of test/train splits in cross validation...
accuracy = 0.5915618390292775
weightedPrecision = 0.6606556463764695
weightedRecall = 0.5915618390292775
f1 = 0.5274611774053655

Run 3 out of 3 of test/train splits in cross validation...
accuracy = 0.5823200788695366
weightedPrecision = 0.692290735649013
weightedRecall = 0.5823200788695366
f1 = 0.5186412846531507


In [30]:
#
# Evaluate average and STD of each metric and print a table
#
import numpy as np
score_averages = defaultdict(float)

# Compute the table data
average_stds = [] # ha
for metric_name in metric_names:
  metric_scores = scores[metric_name]
  
  average_accuracy = sum(metric_scores) / len(metric_scores)
  score_averages[metric_name] = average_accuracy
  
  std_accuracy = np.std(metric_scores)
  
  average_stds.append((metric_name, average_accuracy, std_accuracy))

# Print the table
print("\nExperiment Log")
print("--------------")
print(tabulate(average_stds, headers=["Metric", "Average", "STD"]))


Experiment Log
--------------
Metric               Average         STD
-----------------  ---------  ----------
accuracy            0.584963  0.00469702
weightedPrecision   0.682129  0.0151913
weightedRecall      0.584963  0.00469702
f1                  0.520305  0.0052966


In [33]:
#
# Persist the score to a sccore log that exists between runs
#
import pickle

# Load the score log or initialize an empty one
try:
  score_log_filename = "../models/score_log.pickle"
  score_log = pickle.load(open(score_log_filename, "rb"))
  if not isinstance(score_log, list):
    score_log = []
except IOError:
  score_log = []

# Compute the existing score log entry
score_log_entry = {metric_name: score_averages[metric_name] for metric_name in metric_names}

# Compute and display the change in score for each metric
try:
  last_log = score_log[-1]
except (IndexError, TypeError, AttributeError):
  last_log = score_log_entry

experiment_report = []
for metric_name in metric_names:
  run_delta = score_log_entry[metric_name] - last_log[metric_name]
  experiment_report.append((metric_name, run_delta))

print("\nExperiment Report")
print("-----------------")
print(tabulate(experiment_report, headers=["Metric", "Score"]))

# Append the existing average scores to the log
score_log.append(score_log_entry)

# Persist the log for next run
pickle.dump(score_log, open(score_log_filename, "wb"))


Experiment Report
-----------------
Metric                  Score
-----------------  ----------
accuracy           0.00230508
weightedPrecision  0.0322531
weightedRecall     0.00230508
f1                 0.00333393


In [34]:
#
# Analyze and report feature importance changes
#

# Compute averages for each feature
feature_importance_entry = defaultdict(float)
for feature_name, value_list in feature_importances.items():
  average_importance = sum(value_list) / len(value_list)
  feature_importance_entry[feature_name] = average_importance

# Sort the feature importances in descending order and print
import operator
sorted_feature_importances = sorted(
  feature_importance_entry.items(),
  key=operator.itemgetter(1),
  reverse=True
)

print("\nFeature Importances")
print("-------------------")
print(tabulate(sorted_feature_importances, headers=['Name', 'Importance']))


Feature Importances
-------------------
Name                Importance
----------------  ------------
DepDelay            0.751766
Route_index         0.0529562
DayOfYear_index     0.0433443
DayOfMonth_index    0.0292654
Origin_index        0.0282372
Dest_index          0.0194616
Distance            0.0182077
Carrier_index       0.00928362
CRSDepHourOfDay     0.00847469
DayOfWeek_index     0.00586538
CRSArrHourOfDay     0.00490074


In [36]:
#
# Compare this run's feature importances with the previous run's
#
  
# Load the feature importance log or initialize an empty one
try:
  feature_log_filename = "../models/feature_log.pickle"
  feature_log = pickle.load(open(feature_log_filename, "rb"))
  if not isinstance(feature_log, list):
    feature_log = []
except IOError:
  feature_log = []

# Compute and display the change in score for each feature
try:
  last_feature_log = feature_log[-1]
except (IndexError, TypeError, AttributeError):
  last_feature_log = defaultdict(float)
  for feature_name, importance in feature_importance_entry.items():
    last_feature_log[feature_name] = importance

# Compute the deltas
feature_deltas = {}
for feature_name in feature_importances.keys():
  run_delta = feature_importance_entry[feature_name] - last_feature_log[feature_name]
  feature_deltas[feature_name] = run_delta

# Sort feature deltas, biggest change first
import operator
sorted_feature_deltas = sorted(
  feature_deltas.items(),
  key=operator.itemgetter(1),
  reverse=True
)

# Display sorted feature deltas
print("\nFeature Importance Delta Report")
print("-------------------------------")
print(tabulate(sorted_feature_deltas, headers=["Feature", "Delta"]))

# Append the existing average deltas to the log
feature_log.append(feature_importance_entry)

# Persist the log for next run
pickle.dump(feature_log, open(feature_log_filename, "wb"))


Feature Importance Delta Report
-------------------------------
Feature                 Delta
----------------  -----------
DayOfMonth_index   0.0106545
Route_index        0.0097556
CRSDepHourOfDay    0.00847469
Origin_index       0.00564985
CRSArrHourOfDay    0.00490074
DayOfWeek_index    0.00257626
Carrier_index      0.00222444
DayOfYear_index    0.00166727
Distance           0.00135526
Dest_index        -0.00166013
DepDelay          -0.0512483


### Interpreting Our Results

Interpreting the output, it looks like the combined effect of these fields is to impact feature importance by about 1%, but the effect on accuracy is insignificant. We’ll leave the fields in, although they don’t help much. Without resorting to advanced time series analysis, it seems we’ve milked all we can from date/time-based features.

## Incorporating Airplane Data

Recall from `Investigating Airplanes (Entities)` that we incorporated data on airplane manufacturers into our data model. For instance, we analyzed the distribution of manufacturers in the American commercial fleet. In this section, we’re going to join in airline data and see what impact this has on the model’s accuracy.

I wonder whether properties of the aircraft (called the “metal” of the flight) influence delays? For instance, bigger aircraft fly higher and can go over weather, while smaller aircraft may be less able to do so. I can’t honestly think of a reason why the engine manufacturer, airplane manufacturer, or manufacture year would have an impact on the model, but since we’re importing one field, we may as well try them all! Note that we can simply drop any features that don’t rank as very significant. The beauty of our experimental model with decision trees is that it doesn’t cost extra to try extra fields. Sometimes you can simply let the model decide what matters.

Note that when dealing with team members and with other teams who need an accounting of your time in order to coordinate with you, a description of the experiments you are running will help keep the teams in sync. For instance, “We are attempting to incorporate a new dataset which we scraped from the FAA website into our flight delay predictive model” would make a good experimental description during an agile sprint.

### Extracting Airplane Features

To add airplane features to our model, we need to create a new feature extraction script, [ch09/extract_features_with_airplanes.py](extract_features_with_airplanes.py). We can do this by copying and altering [ch09/extract_features.py](extract_features.py).

First we add `TailNum` to the fields we select from our training data. Because this column also appears in our airplane dataset, we need to name it differently or we won’t easily be able to access the column after the join. We’ll name it `FeatureTailNum`:

In [41]:
# Load the on-time parquet file
input_path = "../data/on_time_performance.parquet"
on_time_dataframe = spark.read.parquet(input_path)
on_time_dataframe.registerTempTable("on_time_performance")

# Select a few features of interest
simple_on_time_features = spark.sql("""
SELECT
  FlightNum,
  FlightDate,
  DayOfWeek,
  DayofMonth AS DayOfMonth,
  CONCAT(Month, '-',  DayofMonth) AS DayOfYear,
  Carrier,
  Origin,
  Dest,
  Distance,
  DepDelay,
  ArrDelay,
  CRSDepTime,
  CRSArrTime,
  CONCAT(Origin, '-', Dest) AS Route,
  TailNum AS FeatureTailNum
FROM on_time_performance
WHERE FlightDate < '2015-02-01'
""")

simple_on_time_features = simple_on_time_features.sample(False, 0.1)

simple_on_time_features.select(
  "FlightNum",
  "FlightDate",
  "FeatureTailNum"
).show(10)

+---------+----------+--------------+
|FlightNum|FlightDate|FeatureTailNum|
+---------+----------+--------------+
|     1080|2015-01-01|        N005AA|
|     1513|2015-01-01|        N008AA|
|     1455|2015-01-01|        N012AA|
|      903|2015-01-01|        N017AA|
|     2413|2015-01-01|        N357AA|
|     1044|2015-01-01|        N389AA|
|     1650|2015-01-01|        N397AA|
|     1382|2015-01-01|        N3AEAA|
|     1266|2015-01-01|        N3AHAA|
|     1525|2015-01-01|        N3AHAA|
+---------+----------+--------------+
only showing top 10 rows



In [51]:
# Filter nulls, they can't help us
filled_on_time_features = simple_on_time_features.where(simple_on_time_features.ArrDelay.isNotNull())
filled_on_time_features = filled_on_time_features.where(filled_on_time_features.DepDelay.isNotNull())
filled_on_time_features.show(5)

+---------+----------+---------+----------+---------+-------+------+----+--------+--------+--------+----------+----------+-------+--------------+
|FlightNum|FlightDate|DayOfWeek|DayOfMonth|DayOfYear|Carrier|Origin|Dest|Distance|DepDelay|ArrDelay|CRSDepTime|CRSArrTime|  Route|FeatureTailNum|
+---------+----------+---------+----------+---------+-------+------+----+--------+--------+--------+----------+----------+-------+--------------+
|     1513|2015-01-01|        4|         1|      1-1|     AA|   ATL| DFW|   731.0|    -2.0|    -7.0|      1045|      1215|ATL-DFW|        N008AA|
|     1455|2015-01-01|        4|         1|      1-1|     AA|   ATL| DFW|   731.0|    -4.0|   -14.0|      0855|      1025|ATL-DFW|        N012AA|
|      903|2015-01-01|        4|         1|      1-1|     AA|   LAX| EGE|   748.0|    -7.0|    16.0|      0830|      1145|LAX-EGE|        N017AA|
|     2413|2015-01-01|        4|         1|      1-1|     AA|   DFW| LAX|  1235.0|   128.0|   129.0|      0800|      0930|DF

In [42]:
# We need to turn timestamps into timestamps, and not strings or numbers
def convert_hours(hours_minutes):
  hours = hours_minutes[:-2]
  minutes = hours_minutes[-2:]
  
  if hours == '24':
    hours = '23'
    minutes = '59'
  
  time_string = "{}:{}:00Z".format(hours, minutes)
  return time_string

def compose_datetime(iso_date, time_string):
  return "{} {}".format(iso_date, time_string)

def create_iso_string(iso_date, hours_minutes):
  time_string = convert_hours(hours_minutes)
  full_datetime = compose_datetime(iso_date, time_string)
  return full_datetime

def create_datetime(iso_string):
  return iso8601.parse_date(iso_string)

def convert_datetime(iso_date, hours_minutes):
  iso_string = create_iso_string(iso_date, hours_minutes)
  dt = create_datetime(iso_string)
  return dt

def day_of_year(iso_date_string):
  dt = iso8601.parse_date(iso_date_string)
  doy = dt.timetuple().tm_yday
  return doy

def alter_feature_datetimes(row):
  
  flight_date = iso8601.parse_date(row['FlightDate'])
  scheduled_dep_time = convert_datetime(row['FlightDate'], row['CRSDepTime'])
  scheduled_arr_time = convert_datetime(row['FlightDate'], row['CRSArrTime'])
  
  # Handle overnight flights
  if scheduled_arr_time < scheduled_dep_time:
    scheduled_arr_time += datetime.timedelta(days=1)
  
  doy = day_of_year(row['FlightDate'])
  
  return {
    'FlightNum': row['FlightNum'],
    'FlightDate': flight_date,
    'DayOfWeek': int(row['DayOfWeek']),
    'DayOfMonth': int(row['DayOfMonth']),
    'DayOfYear': doy,
    'Carrier': row['Carrier'],
    'Origin': row['Origin'],
    'Dest': row['Dest'],
    'Distance': row['Distance'],
    'DepDelay': row['DepDelay'],
    'ArrDelay': row['ArrDelay'],
    'CRSDepTime': scheduled_dep_time,
    'CRSArrTime': scheduled_arr_time,
    'Route': row['Route'],
    'FeatureTailNum': row['FeatureTailNum'],
  }

In [53]:
timestamp_features = filled_on_time_features.rdd.map(alter_feature_datetimes)
timestamp_df = timestamp_features.toDF()
timestamp_df.show(5)

+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+--------------+-------------------+---------+------+-------+
|ArrDelay|         CRSArrTime|         CRSDepTime|Carrier|DayOfMonth|DayOfWeek|DayOfYear|DepDelay|Dest|Distance|FeatureTailNum|         FlightDate|FlightNum|Origin|  Route|
+--------+-------------------+-------------------+-------+----------+---------+---------+--------+----+--------+--------------+-------------------+---------+------+-------+
|    -7.0|2015-01-01 12:15:00|2015-01-01 10:45:00|     AA|         1|        4|        1|    -2.0| DFW|   731.0|        N008AA|2015-01-01 00:00:00|     1513|   ATL|ATL-DFW|
|   -14.0|2015-01-01 10:25:00|2015-01-01 08:55:00|     AA|         1|        4|        1|    -4.0| DFW|   731.0|        N012AA|2015-01-01 00:00:00|     1455|   ATL|ATL-DFW|
|    16.0|2015-01-01 11:45:00|2015-01-01 08:30:00|     AA|         1|        4|        1|    -7.0| EGE|   748.0|        N017AA|2015-01-

/home/ubuntu/spark/python/pyspark/sql/session.py:356: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


### Joining Airplane Data

Next, we load the airplane data and left join it to our features dataset. Note that null is a problematic value for our `StringIndexer`. But we don’t want to discard empty values or rows either, because whether a variable is present or not is something our decision tree model can use to learn. We use `DataFrame.selectExpr` to `COALESCE` our `null` values to the string `'Empty'`. This will get its own index from `StringIndexer` and things will work out well. Also note that we rename `FeatureTailNum` back to `TailNum` for the final output:

In [56]:
# Load airplanes and left join on tail numbers
airplanes_path = "../data/airplanes.json"
airplanes = spark.read.json(airplanes_path)

# Left outer join ensures all feature records remain, with nulls where airplane records are not available
features_with_airplanes = timestamp_df.join(
  airplanes,
  on=timestamp_df.FeatureTailNum == airplanes.TailNum,
  how="left_outer"
)

# Fill in the nulls 'Empty' with COALESCE
features_with_airplanes = features_with_airplanes.selectExpr(
  "FlightNum",
  "FlightDate",
  "DayOfWeek",
  "DayOfMonth",
  "DayOfYear",
  "Carrier",
  "Origin",
  "Dest",
  "Distance",
  "DepDelay",
  "ArrDelay",
  "CRSDepTime",
  "CRSArrTime",
  "Route",
  "FeatureTailNum AS TailNum",
  "COALESCE(EngineManufacturer, 'Empty') AS EngineManufacturer",
  "COALESCE(EngineModel, 'Empty') AS EngineModel",
  "COALESCE(Manufacturer, 'Empty') AS Manufacturer",
  "COALESCE(ManufacturerYear, 'Empty') AS ManufacturerYear",
  "COALESCE(Model, 'Empty') AS Model",
  "COALESCE(OwnerState, 'Empty') AS OwnerState"
)
features_with_airplanes.show(5)

+---------+-------------------+---------+----------+---------+-------+------+----+--------+--------+--------+-------------------+-------------------+-------+-------+------------------+-----------+------------+----------------+-------+----------+
|FlightNum|         FlightDate|DayOfWeek|DayOfMonth|DayOfYear|Carrier|Origin|Dest|Distance|DepDelay|ArrDelay|         CRSDepTime|         CRSArrTime|  Route|TailNum|EngineManufacturer|EngineModel|Manufacturer|ManufacturerYear|  Model|OwnerState|
+---------+-------------------+---------+----------+---------+-------+------+----+--------+--------+--------+-------------------+-------------------+-------+-------+------------------+-----------+------------+----------------+-------+----------+
|     1513|2015-01-01 00:00:00|        4|         1|        1|     AA|   ATL| DFW|   731.0|    -2.0|    -7.0|2015-01-01 10:45:00|2015-01-01 12:15:00|ATL-DFW| N008AA|             Empty|      Empty|       Empty|           Empty|  Empty|     Empty|
|     1455|2015-

In [57]:
# Explicitly sort the data and keep it sorted throughout. Leave nothing to chance.
sorted_features = features_with_airplanes.sort(
  timestamp_df.DayOfYear,
  timestamp_df.Carrier,
  timestamp_df.Origin,
  timestamp_df.Dest,
  timestamp_df.FlightNum,
  timestamp_df.CRSDepTime,
  timestamp_df.CRSArrTime,
)

In [58]:
# Store as a single json file
output_path = "../data/simple_flight_delay_features_airplanes.json"
sorted_features.repartition(1).write.mode("overwrite").json(output_path)